In [1]:
!pip install /kaggle/input/whoosh-wheel-2-7-4/Whoosh-2.7.4-py2.py3-none-any.whl

Processing /kaggle/input/whoosh-wheel-2-7-4/Whoosh-2.7.4-py2.py3-none-any.whl


In [2]:
from pathlib import Path
from typing import Any
import dill
import polars as pl
import whoosh_utils
import random
import os
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.pipeline import make_pipeline

Processing /kaggle/input/whoosh-wheel-2-7-4/Whoosh-2.7.4-py2.py3-none-any.whl
Whoosh is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [3]:
pl.set_random_seed(0)

In [4]:
import gc
from tqdm import tqdm
interactive= False
if interactive:
    pub_num = pl.read_csv('/kaggle/input/uspto-explainable-ai-validation-index/validation/validation_publication_numbers.csv')[:2500]
    test = pub_num #pl.read_csv(comp_data_dir / "test.csv")
    neigbours_small = pl.read_csv('/kaggle/input/uspto-explainable-ai-validation-index/neighbors_small.csv')
    test = test.join(neigbours_small, on="publication_number", how="left")

    all_pub = test.melt().get_column("value").unique()
    gc.collect()


    comp_data_dir = Path("/kaggle/input/uspto-explainable-ai")
    meta = pl.scan_parquet(comp_data_dir / "patent_metadata.parquet")
    meta = (
        meta.with_columns(
            pl.col("publication_date").dt.year().alias("year"),
            pl.col("publication_date").dt.month().alias("month"),
        )
    #.filter(pl.col("publication_date") >= pl.date(1975, 1, 1))
        .rename({"cpc_codes": "cpc"})
        .collect()
    )
    meta = meta.filter(pl.col("publication_number").is_in(all_pub))
    patents = []
    n_unique = meta.select(["year", "month"]).n_unique()
    for (year, month), _ in tqdm(meta.group_by(["year", "month"]), total=n_unique):
        patent_path = comp_data_dir / f"patent_data/{year}_{month}.parquet"
        patent = pl.scan_parquet(patent_path).select(pl.exclude(["claims", "description"]))
        patent = patent.filter(pl.col("publication_number").is_in(all_pub))
        patents.append(patent)
    print('for ended')
    patent = pl.concat(patents) #pl.LazyFrame
    patent = patent.with_columns(
        pl.lit("").alias("claims"),
        pl.lit("").alias("description"),
    )
    meta_with_text = (
        meta.lazy().join(patent, on="publication_number", how="left")
    )
    meta_with_text = meta_with_text.collect(streaming=True)
    print('collected')
    all_pub_nums = neigbours_small[:,1:].melt()[:,1].unique()
    test_meta = (meta_with_text.filter(pl.col('publication_number').is_in(all_pub_nums)).unique(subset=['publication_number']))
    all_pub_nums = test_meta.get_column('publication_number')
    del meta_with_text
    del neigbours_small
    del patent
    del patents
    gc.collect()
else:
    comp_data_dir = Path("/kaggle/input/uspto-explainable-ai")
    test = pl.read_csv(comp_data_dir / "test.csv")
    neigbours = pl.read_csv('/kaggle/input/uspto-explainable-ai/nearest_neighbors.csv')
    test = test.join(neigbours, on="publication_number", how="left")
    all_pub = test.melt().get_column("value").unique()
    gc.collect()


    meta = pl.scan_parquet(comp_data_dir / "patent_metadata.parquet")
    meta = (
        meta.with_columns(
            pl.col("publication_date").dt.year().alias("year"),
            pl.col("publication_date").dt.month().alias("month"),
        )
        .rename({"cpc_codes": "cpc"})
        .collect()
    )
    meta = meta.filter(pl.col("publication_number").is_in(all_pub))
    patents = []
    n_unique = meta.select(["year", "month"]).n_unique()
    for (year, month), _ in tqdm(meta.group_by(["year", "month"]), total=n_unique):
        patent_path = comp_data_dir / f"patent_data/{year}_{month}.parquet"
        patent = pl.scan_parquet(patent_path).select(pl.exclude(["claims", "description"]))
        patent = patent.filter(pl.col("publication_number").is_in(all_pub))
        patents.append(patent)
    print('for ended')
    patent = pl.concat(patents) #pl.LazyFrame
    patent = patent.with_columns(
        pl.lit("").alias("claims"),
        pl.lit("").alias("description"),
    )
    meta_with_text = (
        meta.lazy().join(patent, on="publication_number", how="left")
    )
    meta_with_text = meta_with_text.collect(streaming=True)
    print('collected')
    neigbours = neigbours.filter(neigbours.get_column('publication_number').is_in(test.get_column('publication_number')))
    all_pub_nums = neigbours[:,1:].melt()[:,1].unique()
    test_meta = (meta_with_text.filter(pl.col('publication_number').is_in(all_pub_nums)).unique(subset=['publication_number']))
    del patents
    del neigbours
    del patent
    del meta_with_text
    gc.collect()

/tmp/ipykernel_18/3746791706.py:57: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  all_pub = test.melt().get_column("value").unique()
100%|██████████| 365/365 [00:00<00:00, 576.39it/s]


for ended
collected


/tmp/ipykernel_18/3746791706.py:90: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  all_pub_nums = neigbours[:,1:].melt()[:,1].unique()


In [5]:
import dill

def save_as_pickle(obj, path):
    with open(path, "wb") as f:
        dill.dump(obj, f)
def load_pickle(path):
    with open(path, "rb") as f:
        return dill.load(f)

In [6]:
if interactive:
    test_idx = whoosh_utils.load_index("/kaggle/input/uspto-explainable-ai-validation-index/validation/validation_index")
    searcher = whoosh_utils.get_searcher(test_idx)
    qp = whoosh_utils.get_query_parser()

In [7]:
def handle(ti, frequency_dict):
    BRS_STOPWORDS = set(['an', 'are', 'by', 'for', 'if', 'into', 'is', 'no', 'not', 'of', 'on', 'such',
        'that', 'the', 'their', 'then', 'there', 'these', 'they', 'this', 'to', 'was', 'will'])
    ti_set = set()
    ti_cleaned = re.findall(r'\b\w+\b', ti.lower())
    w_prev = ''
    for w in ti_cleaned:
        if w in BRS_STOPWORDS or len(w) < 2:
            continue
        if w not in ti_set:
            ti_set.add(w)
            if w not in frequency_dict:
                frequency_dict[w] = 1
            else:
                frequency_dict[w] += 1
        if w_prev != '':
            if w_prev + ' ' + w not in ti_set:
                ti_set.add(w_prev + ' ' + w)
                if w_prev + ' ' + w not in frequency_dict:
                    frequency_dict[w_prev + ' ' + w] = 1
                else:
                    frequency_dict[w_prev + ' ' + w] += 1
        w_prev = w

In [8]:
from collections import defaultdict
from itertools import product
def handle_cross(first, second, frequency_dict, first_dict, second_dict):
    BRS_STOPWORDS = {'an', 'are', 'by', 'for', 'if', 'into', 'is', 'no', 'not', 
                     'of', 'on', 'such', 'that', 'the', 'their', 'then', 
                     'there', 'these', 'they', 'this', 'to', 'was', 'will'}

    # Use a single regex findall call and filter in one go
    first_cleaned = {word for word in re.findall(r'\b\w+\b', first.lower()) 
                     if word not in BRS_STOPWORDS and len(word) > 1 and word in first_dict}
    second_cleaned = {word for word in re.findall(r'\b\w+\b', second.lower()) 
                      if word not in BRS_STOPWORDS and len(word) > 1 and word in second_dict}

    for w1, w2 in product(first_cleaned, second_cleaned):
        frequency_dict[(w1, w2)] += 1
        
def handle_single(first, frequency_dict, first_dict):
    BRS_STOPWORDS = {'an', 'are', 'by', 'for', 'if', 'into', 'is', 'no', 'not', 
                     'of', 'on', 'such', 'that', 'the', 'their', 'then', 
                     'there', 'these', 'they', 'this', 'to', 'was', 'will'}

    # Use a single regex findall call and filter in one go
    first_cleaned = {word for word in re.findall(r'\b\w+\b', first.lower()) 
                     if word not in BRS_STOPWORDS and len(word) > 1 and word in first_dict}

    for w1, w2 in product(first_cleaned, first_cleaned):
        frequency_dict[(w1, w2)] += 1

In [9]:
import re
load_dicts = False
if load_dicts:
    desc_freqs = load_pickle('/kaggle/input/claims-freqs-with-bigrams-for-random-state-0/full_desc_freqs_with_bigrams_first_2500.pkl')
    claims_freqs = load_pickle('/kaggle/input/claims-freqs-with-bigrams-for-random-state-0/full_claims_freqs_with_bigrams_first_2500.pkl')
else:
    p_meta = pl.read_parquet(os.path.join(comp_data_dir, "patent_metadata.parquet"), columns=["publication_number", "publication_date", "cpc_codes"])
    p_meta = p_meta.filter(pl.col("publication_number").is_in(all_pub_nums))

    p_meta = p_meta.with_columns(pl.col("publication_date").dt.year().alias("year"))
    p_meta = p_meta.with_columns(pl.col("publication_date").dt.month().alias("month"))
    #claims_pairs_freqs = defaultdict(int)
    desc_freqs = dict()
    claims_freqs = dict()
    c = 0
    for (year, month), meta_df in tqdm(p_meta.group_by(["year", "month"], maintain_order=True)):
        #meta_df = meta_df.with_columns(pl.col("cpc_codes").list.join(" "))
        patents = pl.read_parquet(os.path.join(comp_data_dir, f"patent_data/{year}_{month}.parquet"))
        patents = patents.filter(pl.col("publication_number").is_in(meta_df["publication_number"]))
        for i in range(meta_df.shape[0]):
            d = dict()
            p = patents.filter(pl.col("publication_number") == meta_df[i, "publication_number"])
            if p.shape[0] > 0:
                desc = p[0, "description"]
                #handle_single(desc, desc_pairs_freqs, desc_freqs)
                handle(desc, desc_freqs)
                claims = p[0, "claims"]
                handle(claims, claims_freqs)
                #handle_single(claims, claims_pairs_freqs, claims_freqs)
            
                #handle_cross(claims, desc, claims_desc_freqs, claims_freqs, desc_freqs)
                del desc
                del claims
            del p
        del patents
    del p_meta

361it [19:39,  3.27s/it]


In [10]:
#save_as_pickle(desc_freqs, 'full_desc_freqs_with_bigrams_first_2500.pkl')
#save_as_pickle(claims_freqs, 'full_claims_freqs_with_bigrams_first_2500.pkl')
#save_as_pickle(claims_desc_freqs, 'claims500_desc500_freqs_first_2500.pkl')

In [11]:
claims_freqs = {key: value for key, value in claims_freqs.items() if value <= 100}
desc_freqs = {key: value for key, value in desc_freqs.items() if value <= 100}

In [12]:
import h5py
def extract_claims(pb_year, pb_month, cur_pb):
    if 1790 <= pb_year <= 1999:
        folder = '/kaggle/input/claims1790to1999'
    elif 2000 <= pb_year <= 2011:
        folder = '/kaggle/input/2000-to-2011'
    elif 2012 <= pb_year <= 2023:
        folder = '/kaggle/input/claims2012to2023'
    
    with h5py.File(f'{folder}/{pb_year}_{pb_month}.h5', 'r') as h5:
        if cur_pb in h5:
            data = h5[cur_pb][:]
            h5_detd = data.tobytes().decode()
        else:
            h5_detd = ''
    if h5_detd == None:
        h5_detd = ''
    return h5_detd

def extract_desc(pb_year, pb_month, cur_pb):
    if 1975 <= pb_year <= 1994:
        folder = '/kaggle/input/1975up1994'
    elif 1995 <= pb_year <= 2004:
        folder = '/kaggle/input/1995up2004'
    elif 2005 <= pb_year <= 2009:
        folder = '/kaggle/input/2005up2009'
    elif 1700 <= pb_year <= 1929:
        folder = '/kaggle/input/1700up1929'
    elif 1930 <= pb_year <= 1974:
        folder = '/kaggle/input/1930up1974'
    elif 2010 <= pb_year <= 2013:
        folder = '/kaggle/input/2010up2013'
    elif 2014 <= pb_year <= 2016:
        folder = '/kaggle/input/2014up2016'
    elif 2017 <= pb_year <= 2019:
        folder = '/kaggle/input/2017up2019'
    elif 2020 <= pb_year <= 2021:
        folder = '/kaggle/input/2020up2021'
    elif pb_year == 2022:
        folder = '/kaggle/input/2022up2022'
    elif pb_year == 2023:
        folder = '/kaggle/input/2023up2023'
    
    with h5py.File(f'{folder}/{pb_year}_{pb_month}.h5', 'r') as h5:
        if cur_pb in h5:
            data = h5[cur_pb][:]
            h5_detd = data.tobytes().decode()
        else:
            h5_detd = ''
    if h5_detd == None:
        h5_detd = ''
    return h5_detd

In [13]:
from numpy.typing import NDArray
from scipy.sparse import coo_matrix

def ap50(preds: list[str], labels: list[str]) -> float:
    precisions = list()
    n_found = 0
    for e, i in enumerate(preds):
        if i in labels:
            n_found += 1
        precisions.append(n_found / (e + 1))  # this is the line that is probably incorrect for competition
    return sum(precisions) / 50

def count_intersection(cand, target):
    return set(cand).intersection(set(target))

In [14]:
def execute_query(query: str, qp, searcher, target, results_limit=50) -> list:
    if len(query) > 10_000:
        raise ValueError('Query length at exceeds 10,000 characters.')
    if 'id:' in query:
        raise ValueError('Searching for specific patent IDs is banned.')

    to_search = qp.parse(query)
    results = searcher.search(to_search, limit=results_limit)
    results = [x['id'] for x in results]
    #print([i for i in range(len(results)) if results[i] in target])
    assert len(results) <= results_limit
    return results

In [15]:
def get_year_and_month(single_pb, patent_metadata):
    pb_meta = patent_metadata.filter(patent_metadata.get_column('publication_number') == single_pb)
    pb_pub_date = pb_meta.get_column('publication_date')[0]
    pb_year, pb_month = pb_pub_date.year, pb_pub_date.month
    return pb_year, pb_month

In [16]:
def get_claims_and_desc_frame(target, test_meta):
    frame = pl.DataFrame()
    for tar in target:
        year, month = get_year_and_month(tar, test_meta)
        cur_claim = extract_claims(year, month, tar)
        cur_desc = extract_desc(year, month, tar)
        new_frame = pl.DataFrame({'claims': [cur_claim], 'desc': [cur_desc], 'publication_number': tar})
        frame = frame.vstack(new_frame)
    return frame

In [17]:
import re
def get_ti_ab_freq_dicts(meta_i, col1, col2, pair_frequency_dict, field1, field2):
    rare_words = dict()
    rare_freqs = dict()
    BRS_STOPWORDS = set(['an', 'are', 'by', 'for', 'if', 'into', 'is', 'no', 'not', 'of', 'on', 'such',
        'that', 'the', 'their', 'then', 'there', 'these', 'they', 'this', 'to', 'was', 'will'])
    NUMBER_REGEX = re.compile(r'^(\d+|\d{1,3}(,\d{3})*)(\.\d+)?$')
    rare_dicts = dict()
    titles = meta_i.get_column(col1)
    abss = meta_i.get_column(col2)
    pub_nums = meta_i.get_column('publication_number')
    for (ti, ab, pub_num) in zip(titles, abss, pub_nums):
        ti_cleaned = re.findall(r'\b\w+\b', ti.lower())
        ab_cleaned = re.findall(r'\b\w+\b', ab.lower())
        cur_rare_dict = dict()
        cur_used_set = set()
        for w1 in ti_cleaned:
            if w1 in BRS_STOPWORDS or len(w1) < 2 or NUMBER_REGEX.match(w1):
                continue
            for w2 in ab_cleaned:
                if w2 in BRS_STOPWORDS or len(w2) < 2 or NUMBER_REGEX.match(w2):
                    continue
                w = (w1, w2)
                if pair_frequency_dict[w] < 50 and w not in cur_used_set:
                    cur_used_set.add(w)
                    rare_words[w] = pair_frequency_dict[w]
                    cur_rare_dict[field1 + ':' + w[0] + ' ' + field2 + ':' + w[1]] = pair_frequency_dict[w]
                    if w in rare_freqs:
                        rare_freqs[w] += 1
                    else:
                        rare_freqs[w] = 1
        
        rare_dicts[pub_num] = cur_rare_dict
    ideal_match = []
    #print(rare_freqs)
    for key, value in rare_freqs.items():
        if rare_words[key] == value:
            ideal_match.append(key)
    return rare_dicts, set(ideal_match)

def create_pairs_ti_ab(test_meta, col1, col2):
    BRS_STOPWORDS = set(['an', 'are', 'by', 'for', 'if', 'into', 'is', 'no', 'not', 'of', 'on', 'such',
        'that', 'the', 'their', 'then', 'there', 'these', 'they', 'this', 'to', 'was', 'will'])
    titles = test_meta.get_column(col1).fill_null("")
    abss = test_meta.get_column(col2).fill_null("")

    frequency_dict = dict()
    for (ti, ab) in zip(titles, abss):
        ti_set = set()
        ti_cleaned = re.findall(r'\b\w+\b', ti.lower())
        ab_cleaned = re.findall(r'\b\w+\b', ab.lower())
        for w in ti_cleaned:
            if w in BRS_STOPWORDS or len(w) < 2:
                continue
            for w2 in ab_cleaned:
                if w2 in BRS_STOPWORDS or len(w) < 2:
                    continue
                if (w, w2) not in ti_set:
                    ti_set.add((w, w2))
                else:
                    continue
                if (w, w2) not in frequency_dict:
                    frequency_dict[w, w2] = 1
                else:
                    frequency_dict[w, w2] += 1
    return frequency_dict

In [18]:
import re
def create_pairs_diff_cols(test_meta, col1, col2):
    BRS_STOPWORDS = set(['an', 'are', 'by', 'for', 'if', 'into', 'is', 'no', 'not', 'of', 'on', 'such',
        'that', 'the', 'their', 'then', 'there', 'these', 'they', 'this', 'to', 'was', 'will'])
    titles = test_meta.get_column(col1).fill_null("")
    cpcs = test_meta.get_column(col2)
    NUMBER_REGEX = re.compile(r'^(\d+|\d{1,3}(,\d{3})*)(\.\d+)?$')
    frequency_dict = dict()
    for (ti, cur_cpc) in zip(titles, cpcs):
        ti_set = set()
        ti_cleaned = re.findall(r'\b\w+\b', ti.lower())

        for w in ti_cleaned:
            if w in BRS_STOPWORDS or len(w) < 2 or NUMBER_REGEX.match(w):
                continue
            for w2 in cur_cpc:
                if w2 in BRS_STOPWORDS or len(w) < 2:
                    continue
                if (w, w2) not in ti_set:
                    ti_set.add((w, w2))
                else:
                    continue
                if (w, w2) not in frequency_dict:
                    frequency_dict[w, w2] = 1
                else:
                    frequency_dict[w, w2] += 1
    return frequency_dict
ti_cpc_pairs_frequency_dict = create_pairs_diff_cols(test_meta, 'title', 'cpc')
ab_cpc_pairs_frequency_dict = create_pairs_diff_cols(test_meta, 'abstract', 'cpc')
ti_ab_pairs_frequency_dict = create_pairs_ti_ab(test_meta, 'title', 'abstract')

In [19]:
def get_diff_col_pair_freq_dicts(meta_i, col1, col2, pair_frequency_dict, field1, field2):
    rare_words = dict()
    rare_freqs = dict()
    BRS_STOPWORDS = set(['an', 'are', 'by', 'for', 'if', 'into', 'is', 'no', 'not', 'of', 'on', 'such',
        'that', 'the', 'their', 'then', 'there', 'these', 'they', 'this', 'to', 'was', 'will'])
    NUMBER_REGEX = re.compile(r'^(\d+|\d{1,3}(,\d{3})*)(\.\d+)?$')
    rare_dicts = dict()
    titles = meta_i.get_column(col1)
    cpcs = meta_i.get_column(col2)
    pub_nums = meta_i.get_column('publication_number')
    for (ti, cpc_cur, pub_num) in zip(titles, cpcs, pub_nums):
        ti_cleaned = re.findall(r'\b\w+\b', ti.lower())
        cur_rare_dict = dict()
        cur_used_set = set()
        for w1 in ti_cleaned:
            if w1 in BRS_STOPWORDS or len(w1) < 2 or NUMBER_REGEX.match(w1):
                continue
            for w2 in cpc_cur:
                if w2 in BRS_STOPWORDS or len(w2) < 2 or NUMBER_REGEX.match(w2):
                    continue
                w = (w1, w2)
                if pair_frequency_dict[w] < 50 and w not in cur_used_set:
                    cur_used_set.add(w)
                    rare_words[w] = pair_frequency_dict[w]
                    cur_rare_dict[field1 + ':' + w[0] + ' ' + field2 + ':' + w[1]] = pair_frequency_dict[w]
                    if w in rare_freqs:
                        rare_freqs[w] += 1
                    else:
                        rare_freqs[w] = 1
        
        rare_dicts[pub_num] = cur_rare_dict
    ideal_match = []
    #print(rare_freqs)
    for key, value in rare_freqs.items():
        if rare_words[key] == value:
            ideal_match.append(key)
    return rare_dicts, set(ideal_match)

In [20]:
def mscr_new(sets, token_counter):
    mscr_set = set()
    #global tokens_for_mscr
    #sorted_sets = sorted(sets, key=len)
    retain_fr = dict()
    for s in sets:
        for el in s:
            if el in retain_fr:
                retain_fr[el] += 1
            else:
                retain_fr[el] = 1
    retain_freqs = list(dict(sorted(retain_fr.items(), key=lambda item: item[1], reverse=True)).keys())

    if token_counter == 49:
        for i in range(len(retain_freqs)):
            if ' ' not in retain_freqs[i]:
                return [retain_freqs[i]]
        return []
    if len(retain_freqs) == 0:
        return []
    max_freq = retain_fr[retain_freqs[0]]
    
    next_to_add = retain_freqs[0]
    for i in range(len(retain_freqs)):
        if retain_fr[retain_freqs[i]] == max_freq:
            if ' ' not in retain_freqs[i]:
                next_to_add = retain_freqs[i]
                break
    token_counter += len(next_to_add.split()) + 1
    if token_counter >= 50:
        return [next_to_add]
    pruned_set = []
    for s in sets:
        if next_to_add not in s and len(s) > 0:
            pruned_set.append(s)
    if any(len(s) > 0 for s in sets):
        return [next_to_add] + mscr_new(pruned_set, token_counter)
    else:
        return []

In [21]:
import re
from itertools import combinations
def create_pairs_dict(test_meta, col):
    BRS_STOPWORDS = set(['an', 'are', 'by', 'for', 'if', 'into', 'is', 'no', 'not', 'of', 'on', 'such',
        'that', 'the', 'their', 'then', 'there', 'these', 'they', 'this', 'to', 'was', 'will'])
    titles = test_meta.get_column(col).fill_null("")
    frequency_dict = dict()
    for ti in tqdm(titles):
        ti_set = set()
        ti_cleaned = re.findall(r'\b\w+\b', ti.lower())
        ti_cleaned = sorted([token for token in ti_cleaned if not (token in BRS_STOPWORDS or len(token) < 2)])
        all_pairs = combinations(ti_cleaned, 2)
        for pair in all_pairs:
            if (pair[0], pair[1]) in frequency_dict:
                frequency_dict[(pair[0], pair[1])] += 1
            else:
                frequency_dict[(pair[0], pair[1])] = 1
    return frequency_dict

ti_pairs_frequency_dict = create_pairs_dict(test_meta, 'title')
#ab_pairs_frequency_dict = create_pairs_dict(test_meta, 'abstract')

100%|██████████| 500/500 [00:00<00:00, 38442.47it/s]


In [22]:
def get_pair_freq_dicts(meta_i, col, pair_frequency_dict, field):
    rare_words = dict()
    rare_freqs = dict()
    BRS_STOPWORDS = set(['an', 'are', 'by', 'for', 'if', 'into', 'is', 'no', 'not', 'of', 'on', 'such',
        'that', 'the', 'their', 'then', 'there', 'these', 'they', 'this', 'to', 'was', 'will'])
    NUMBER_REGEX = re.compile(r'^(\d+|\d{1,3}(,\d{3})*)(\.\d+)?$')
    rare_dicts = dict()
    titles = meta_i.get_column(col)
    pub_nums = meta_i.get_column('publication_number')
    for ti, pub_num in zip(titles, pub_nums):
        ti_cleaned = re.findall(r'\b\w+\b', ti.lower())
        ti_cleaned = sorted([token for token in ti_cleaned if not (token in BRS_STOPWORDS or len(token) < 2)])
        cur_rare_dict = dict()
        cur_used_set = set()
        all_pairs = combinations(ti_cleaned, 2)
        for w in all_pairs:
            if NUMBER_REGEX.match(w[0]) or NUMBER_REGEX.match(w[1]):
                continue
            if pair_frequency_dict[w] < 50:
                rare_words[w] = pair_frequency_dict[w]
                cur_rare_dict[field + ':' + w[0] + ' ' + field + ':' + w[1]] = pair_frequency_dict[w]
                if w in rare_freqs:
                    rare_freqs[w] += 1
                else:
                    rare_freqs[w] = 1
        rare_dicts[pub_num] = cur_rare_dict
    ideal_match = []
    #print(rare_freqs)
    for key, value in rare_freqs.items():
        if rare_words[key] == value: #*2
            ideal_match.append(key)
    return rare_dicts, set(ideal_match)

In [23]:
def get_freq_dicts(meta_i, col, frequency_dict, field):
    rare_words = dict()
    rare_freqs = dict()
    full_cover = dict()
    good_cover = []
    BRS_STOPWORDS = set(['an', 'are', 'by', 'for', 'if', 'into', 'is', 'no', 'not', 'of', 'on', 'such',
        'that', 'the', 'their', 'then', 'there', 'these', 'they', 'this', 'to', 'was', 'will'])
    NUMBER_REGEX = re.compile(r'^(\d+|\d{1,3}(,\d{3})*)(\.\d+)?$')
    rare_dicts = dict()
    titles = meta_i.get_column(col)
    pub_nums = meta_i.get_column('publication_number')
    all_used_sets = dict()
    for (ti, pub_num) in zip(titles, pub_nums):
        ti_cleaned = re.findall(r'\b\w+\b', ti.lower())
        w_prev = ''
        cur_rare_dict = dict()
        cur_used_set = set()
        for w in ti_cleaned:
            if w in BRS_STOPWORDS or len(w) < 2:
                continue
            if NUMBER_REGEX.match(w):
                w_prev = ''
                continue
            
            if w in frequency_dict and w not in cur_used_set and frequency_dict[w] <= 50:        #ограничение на 50
                rare_words[w] = frequency_dict[w]
                cur_rare_dict[field + ':' + w] = frequency_dict[w]
                if w in rare_freqs:
                    rare_freqs[w] += 1
                else:
                    rare_freqs[w] = 1
            if w not in cur_used_set:
                if w in full_cover:
                    full_cover[w] += 1
                else:
                    full_cover[w] = 1
                    
            if w_prev != '' and w_prev + ' ' + w in frequency_dict and w_prev + ' ' + w not in cur_used_set and frequency_dict[w_prev + ' ' + w] <= 50:
                rare_words[w_prev + ' ' + w] = frequency_dict[w_prev + ' ' + w]
                cur_rare_dict[field + ':"' + w_prev + ' ' + w + '"'] = frequency_dict[w_prev + ' ' + w]
                if w_prev + ' ' + w in rare_freqs:
                    rare_freqs[w_prev + ' ' + w] += 1
                else:
                    rare_freqs[w_prev + ' ' + w] = 1
                    
            if w_prev != '' and w_prev + ' ' + w not in cur_used_set:
                if w_prev + ' ' + w in full_cover:
                    full_cover[w_prev + ' ' + w] += 1
                else:
                    full_cover[w_prev + ' ' + w] = 1
            
            cur_used_set.add(w)
            cur_used_set.add(w_prev + ' ' + w)
            w_prev = w
        #cur_rare_dict = dict(sorted(cur_rare_dict.items(), key=lambda item: item[1], reverse=True))
        all_used_sets[pub_num] = cur_used_set
        rare_dicts[pub_num] = cur_rare_dict
    ideal_match = []
    thres = 1
    val_thres = 1
    if field == 'ab':
        val_thres = 5
    if field == 'detd' or field == 'clm':
        thres = 1
        val_thres = 10
    #print(rare_freqs)
    for key, value in rare_freqs.items():
        if rare_words[key] == value:
            ideal_match.append(key)
            #good_cover.append(key)
        #elif value / rare_words[key] >= thres and value >= val_thres:
            #good_cover.append(key)
    #cover_dicts = dict()
    '''
    for pub_num in pub_nums:
        cover_dicts[pub_num] = set()
    for word in good_cover:
        for pub_num, cur_used_set in all_used_sets.items():
            if word in cur_used_set:
                cover_dicts[pub_num].add(word)'''
    return rare_dicts, set(ideal_match), full_cover#, good_cover, cover_dicts

In [24]:
def create_freq_dict_claims(claims, col):
    BRS_STOPWORDS = set(['an', 'are', 'by', 'for', 'if', 'into', 'is', 'no', 'not', 'of', 'on', 'such',
        'that', 'the', 'their', 'then', 'there', 'these', 'they', 'this', 'to', 'was', 'will'])
    titles=claims
    frequency_dict = dict()
    for ti in titles:
        ti_dict = set()
        ti_cleaned = re.findall(r'\b\w+\b', ti.lower())
        w_prev = ''
        for w in ti_cleaned:
            if w in BRS_STOPWORDS or len(w) < 2:
                continue
            if w not in ti_set:
                ti_set.add(w)
                if w not in frequency_dict:
                    frequency_dict[w] = 1
                else:
                    frequency_dict[w] += 1
            if w_prev != '':
                if w_prev + ' ' + w not in ti_set:
                    ti_set.add(w_prev + ' ' + w)
                    if w_prev + ' ' + w not in frequency_dict:
                        frequency_dict[w_prev + ' ' + w] = 1
                    else:
                        frequency_dict[w_prev + ' ' + w] += 1
            w_prev = w
    return frequency_dict

In [25]:
import re
def create_cpc_dict(test_meta):
    titles = test_meta.get_column('cpc')
    frequency_dict = dict()
    for ti in titles:
        for w in ti:
            if w not in frequency_dict:
                frequency_dict[w] = 1
            else:
                frequency_dict[w] += 1
    return frequency_dict
def create_cpc_pair_dict(test_meta):
    titles = test_meta.get_column('cpc')
    frequency_dict = dict()
    for ti in titles:
        ti_set = set()
        for w in ti:
            for w2 in ti:
                if w2 == w:
                    continue
                if (w, w2) not in ti_set:
                    if (w2, w) not in ti_set:
                        ti_set.add((w, w2))
                if (w, w2) not in frequency_dict:
                    if (w2, w) not in frequency_dict:
                        frequency_dict[w2, w] = 1
                    else:
                        frequency_dict[w2, w] += 1
                else:
                    frequency_dict[w, w2] += 1
    return frequency_dict
cpc_frequency_dict = create_cpc_dict(test_meta)
cpc_pair_frequency_dict = create_cpc_pair_dict(test_meta)

In [26]:
def get_cpc_freq_dicts(meta_i, pair_frequency_dict):
    rare_words = dict()
    rare_freqs = dict()
    rare_dicts = dict()
    full_cover = dict()
    good_cover = []
    cpcs = meta_i.get_column('cpc')
    pub_nums = meta_i.get_column('publication_number')
    all_used_sets = dict()
    for (ti, pub_num) in zip(cpcs, pub_nums):
        cur_rare_dict = dict()
        cur_used_set = set()
        for w in ti:
            if w in cur_used_set:
                continue
            if w in full_cover:
                full_cover[w] += 1
            else:
                full_cover[w] = 1
            if pair_frequency_dict[w] < 50:
                cur_used_set.add(w)
                rare_words[w] = pair_frequency_dict[w]
                cur_rare_dict['cpc:' + w] = pair_frequency_dict[w]
                if w in rare_freqs:
                    rare_freqs[w] += 1
                else:
                    rare_freqs[w] = 1
        all_used_sets[pub_num] = cur_used_set
        
        rare_dicts[pub_num] = cur_rare_dict
    ideal_match = []
    thres = 1
    #print(rare_freqs)
    for key, value in rare_freqs.items():
        if rare_words[key] == value and value >= 1:
            ideal_match.append(key)
            #good_cover.append(key)
        #elif value / rare_words[key] >= thres and value >= 1:
            #good_cover.append(key)
            
            
    '''
    cover_dicts = dict()
    
    for pub_num in pub_nums:
        cover_dicts[pub_num] = set()
    for word in good_cover:
        for pub_num, cur_used_set in all_used_sets.items():
            if word in cur_used_set:
                cover_dicts[pub_num].add(word)'''
    return rare_dicts, set(ideal_match), full_cover#, good_cover, cover_dicts

def get_cpc_pair_freq_dicts(meta_i, pair_frequency_dict):
    rare_words = dict()
    rare_freqs = dict()
    rare_dicts = dict()
    titles = meta_i.get_column('cpc')
    pub_nums = meta_i.get_column('publication_number')
    for (ti, pub_num) in zip(titles, pub_nums):
        cur_rare_dict = dict()
        cur_used_set = set()
        for w1 in ti:
            for w2 in ti:
                if w1 == w2:
                    continue
                if (w1, w2) in pair_frequency_dict:
                    w = (w1, w2)
                else:
                    w = (w2, w1)
                if pair_frequency_dict[w] < 50 and w not in cur_used_set:
                    cur_used_set.add(w)
                    rare_words[w] = pair_frequency_dict[w]
                    cur_rare_dict['cpc:' + w[0] + ' ' + 'cpc:' + w[1]] = pair_frequency_dict[w]
                    if w in rare_freqs:
                        rare_freqs[w] += 1
                    else:
                        rare_freqs[w] = 1
        
        rare_dicts[pub_num] = cur_rare_dict
    ideal_match = []
    #print(rare_freqs)
    for key, value in rare_freqs.items():
        if rare_words[key] == value * 2:
            ideal_match.append(key)
    return rare_dicts, set(ideal_match)

In [27]:
import re
def create_freq_dict(test_meta, col):
    BRS_STOPWORDS = set(['an', 'are', 'by', 'for', 'if', 'into', 'is', 'no', 'not', 'of', 'on', 'such',
        'that', 'the', 'their', 'then', 'there', 'these', 'they', 'this', 'to', 'was', 'will'])
    titles = test_meta.get_column(col).fill_null("")
    frequency_dict = dict()
    for ti in titles:
        ti_set = set()
        ti_cleaned = re.findall(r'\b\w+\b', ti.lower())
        w_prev = ''
        for w in ti_cleaned:
            if w in BRS_STOPWORDS or len(w) < 2:
                continue
            if w not in ti_set:
                ti_set.add(w)
                if w not in frequency_dict:
                    frequency_dict[w] = 1
                else:
                    frequency_dict[w] += 1
            if w_prev != '':
                if w_prev + ' ' + w not in ti_set:
                    ti_set.add(w_prev + ' ' + w)
                    if w_prev + ' ' + w not in frequency_dict:
                        frequency_dict[w_prev + ' ' + w] = 1
                    else:
                        frequency_dict[w_prev + ' ' + w] += 1
            w_prev = w
    return frequency_dict
ti_frequency_dict = create_freq_dict(test_meta, 'title')
ab_frequency_dict = create_freq_dict(test_meta, 'abstract')

In [28]:
def mscr_greedy(sets, token_counter, used_set):
    mscr_set = set()
    retain_fr = dict()
    for s in sets:
        for el in s:
            if el in retain_fr and el not in used_set:
                retain_fr[el] += 1
            else:
                retain_fr[el] = 1
    retain_freqs = list(dict(sorted(retain_fr.items(), key=lambda item: item[1], reverse=True)).keys())
    freqs_iter = 0
    answ = []
    length = len(retain_freqs)
    while token_counter <= 46 and freqs_iter < length:
        next_to_add = retain_freqs[freqs_iter]
        cur_t = 3
        if ' ' in next_to_add[0]:
            cur_t += len(next_to_add[0].split()) - 1
        if ' ' in next_to_add[1]:
            cur_t += len(next_to_add[1].split()) - 1
        token_counter += cur_t
        answ.append(next_to_add)
        freqs_iter += 1
    if token_counter >= 47:
        for i in range(freqs_iter, len(retain_freqs)):
            next_to_add = retain_freqs[i]
            cur_t = 3
            if ' ' in next_to_add[0]:
                cur_t += len(next_to_add[0].split()) - 1
            if ' ' in next_to_add[1]:
                cur_t += len(next_to_add[1].split()) - 1
            if 50 - token_counter >= cur_t:
                answ.append(next_to_add)
                break
                
    return answ

In [29]:
ti_freqs = dict()
ab_freqs = dict()
cpc_freqs = dict()
for i in tqdm(range(len(test_meta))):
    p = test_meta[i]
    abstract = p[0, "abstract"]
    if abstract != None:
        handle(abstract, ab_freqs)
        
    title = p[0, "title"]
    if title != None:
        handle(title, ti_freqs)
        
    cpc = p[0, "cpc"]
    for cp in cpc:
        if cp in cpc_freqs:
            cpc_freqs[cp] += 1
        else:
            cpc_freqs[cp] = 1

100%|██████████| 500/500 [00:00<00:00, 4327.50it/s]


In [30]:
def mscr_mod(sets, token_counter):
    mscr_set = set()
    #sorted_sets = sorted(sets, key=len)
    retain_fr = dict()
    for s in sets:
        for el in s:
            if el in retain_fr:
                retain_fr[el] += 1
            else:
                retain_fr[el] = 1
    if len(retain_fr) == 0:
        return []
    next_to_add = max(retain_fr, key=retain_fr.get)
    '''
    retain_freqs = list(dict(sorted(retain_fr.items(), key=lambda item: item[1], reverse=True)).keys())

    if len(retain_freqs) == 0:
        return []
    max_freq = retain_fr[retain_freqs[0]]
    next_to_add = retain_freqs[0]
    for i in range(len(retain_freqs)):
        if retain_fr[retain_freqs[i]] == max_freq:
            if ' ' not in retain_freqs[i]:
                next_to_add = retain_freqs[i]
                break'''
    cur_t = 3
    if ' ' in next_to_add[0]:
        cur_t += len(next_to_add[0].split()) - 1
    if ' ' in next_to_add[1]:
        cur_t += len(next_to_add[1].split()) - 1
    if token_counter >= 46:
        retain_freqs = list(dict(sorted(retain_fr.items(), key=lambda item: item[1], reverse=True)).keys())
        for token in retain_freqs:
            spl = 0
            for t_part in token:
                spl += len(t_part.split())
            #print('split', spl)
            if spl <= 50 - token_counter:
                return [token]
        return []
    token_counter += cur_t
    #print(token_counter)
    pruned_set = []
    for s in sets:
        if next_to_add not in s and len(s) > 0:
            pruned_set.append(s)
    if any(len(s) > 0 for s in sets):
        return [next_to_add] + mscr_mod(pruned_set, token_counter)
    else:
        return []

In [31]:
def handle_desc_and_claims_pairs(frame, claims_freqs, desc_freqs):
    cur_claims_desc = defaultdict(int)
    BRS_STOPWORDS = {'an', 'are', 'by', 'for', 'if', 'into', 'is', 'no', 'not', 
                     'of', 'on', 'such', 'that', 'the', 'their', 'then', 
                     'there', 'these', 'they', 'this', 'to', 'was', 'will'}
    claims_col = frame.get_column('claims')
    desc_col = frame.get_column('desc')
    for i in range(len(frame)):
        first = claims_col[i]
        second = desc_col[i]
        first_cleaned = {word for word in re.findall(r'\b\w+\b', first.lower()) 
                         if word not in BRS_STOPWORDS and len(word) > 1 and word in claims_freqs}
        second_cleaned = {word for word in re.findall(r'\b\w+\b', second.lower()) 
                          if word not in BRS_STOPWORDS and len(word) > 1 and word in desc_freqs}

        for w1, w2 in product(first_cleaned, second_cleaned):
            cur_claims_desc[(w1, w2)] += 1
    rare_pairs = dict()
    for key, value in cur_claims_desc.items():
        if key in claims_desc_freqs and claims_desc_freqs[key] == value and value > 1:
            rare_pairs[key] = value
    return cur_claims_desc, rare_pairs

In [32]:
import re
def postprocess(min_set):
    min_set = set(min_set)
    while True:
        first = ''
        second = ''
        new = ''
        for s1 in min_set:
            el0 = re.sub(r'[()]', '', s1[0])
            el1 = re.sub(r'[()]', '', s1[1])
            for s2 in min_set:
                
                if s1 == s2:
                    continue
                el3 = re.sub(r'[()]', '', s2[0])
                el4 = re.sub(r'[()]', '', s2[1])
                if el0 == el3 and el0 != '':
                    first = s1
                    second = s2
                    new = ('((' + el0 + ')) ','((' + el1 + ' OR ' + el4 + '))')
                elif el1 == el3 and el1 != '':
                    first = s1
                    second = s2
                    new = ('((' + el1 + ')) ','((' + el0 + ' OR ' + el4 + '))')
                elif el0 == el4 and el0 != '':
                    first = s1
                    second = s2
                    new = ('((' + el0 + ')) ','((' + el1 + ' OR ' + el3 + '))')
                elif el1 == el4 and el1 != '':
                    first = s1
                    second = s2
                    new = ('((' + el1 + ')) ','((' + el0 + ') OR (' + el3 + '))')
        if new == '':
            break
        else:
            min_set -= {first}
            min_set -= {second}
            min_set.add(new)
    return min_set

In [33]:
import numpy as np
import re
import time
from tqdm import tqdm
k=12
scores = []
results = []
ex_time = 0
ban_set = {'in', 'when', 'can', 'through', 'may', 'also', 'using', 'use', 'provides', 'provided', 'includes', 'include', 'including', 'within', 'with', 'or', 'as', 'which', 'so', 'and', 'has', 'have', 'having', 'from', 'at', 'fig', 'invention', 'it', 'be', 'method', 'base', 'one'}
for i in tqdm(range(len(test))):
    target = test[i].to_numpy().flatten()[1:].tolist()
    meta_i = test_meta.filter(pl.col("publication_number").is_in(target))
    st = time.time()
    #print(meta_i)
    
    for j in range(len(meta_i)):
        if meta_i[j, 'title'] == None:
            meta_i[j, 'title'] = ''
        if meta_i[j, 'abstract'] == None:
            meta_i[j, 'abstract'] = ''
          
    rare_ti_cpc_pair_dicts, ti_cpc_pair_ideal_match = get_diff_col_pair_freq_dicts(meta_i, 'title', 'cpc', ti_cpc_pairs_frequency_dict, 'ti', 'cpc')
    rare_ab_cpc_pair_dicts, ab_cpc_pair_ideal_match = get_diff_col_pair_freq_dicts(meta_i, 'abstract', 'cpc', ab_cpc_pairs_frequency_dict, 'ab', 'cpc')
    rare_ti_ab_pair_dicts, ti_ab_pair_ideal_match = get_ti_ab_freq_dicts(meta_i, 'title', 'abstract', ti_ab_pairs_frequency_dict, 'ti', 'ab')
    
    rare_cpc_pair_dicts, cpc_pair_ideal_match = get_cpc_pair_freq_dicts(meta_i, cpc_pair_frequency_dict)
    rare_ti_pair_dicts, ti_pair_ideal_match = get_pair_freq_dicts(meta_i, 'title', ti_pairs_frequency_dict, 'ti')
    #rare_ab_pair_dicts, ab_pair_ideal_match = get_pair_freq_dicts(meta_i, 'abstract', ab_pairs_frequency_dict, 'ab') #bad
    rare_cpc_dicts, cpc_ideal_match, cpc_full_cover = get_cpc_freq_dicts(meta_i, cpc_frequency_dict)
    
    rare_ti_dicts, ti_ideal_match, ti_full_cover = get_freq_dicts(meta_i, 'title', ti_frequency_dict, 'ti')
    
    rare_ab_dicts, ab_ideal_match, ab_full_cover = get_freq_dicts(meta_i, 'abstract', ab_frequency_dict, 'ab')
    claims_or_desc_frame = get_claims_and_desc_frame(target, test_meta)
    rare_clm_dicts, clm_ideal_match, claims_full_cover = get_freq_dicts(claims_or_desc_frame, 'claims', claims_freqs, 'clm')
    rare_desc_dicts, desc_ideal_match, desc_full_cover = get_freq_dicts(claims_or_desc_frame, 'desc', desc_freqs, 'detd')
    
    st = time.time()

    max_len = 0
    merged_list = []
    for j in meta_i.get_column('publication_number'):
        merged_dict = dict()
        
        for key, value in rare_ti_pair_dicts[j].items():
            if value <= 1:
                continue
            first, second = key.split()
            mod_key = (first[3:], second[3:])
            query_key = '(' + first + ' ' + second + ')'
            if mod_key in ti_pair_ideal_match and (ti_freqs[first[3:]] not in ban_set) and (ti_freqs[second[3:]] not in ban_set):
                merged_dict[query_key] = value
        
        for key, value in rare_ti_ab_pair_dicts[j].items():
            if value <= 1:
                continue
            first, second = key.split()
            mod_key = (first[3:], second[3:])
            query_key = '(' + first + ' ' + second + ')'
            if mod_key in ti_ab_pair_ideal_match and (mod_key[0] not in ban_set) and (mod_key[1] not in ban_set):
                merged_dict[query_key] = value
                
        for key, value in rare_ti_cpc_pair_dicts[j].items():
            if value <= 1:
                continue
            first, second = key.split()
            mod_key = (first[3:], second[4:])
            query_key = '(' + first + ' ' + second + ')'
            #print(rare_ti_cpc_pair_dicts[j])
            
            if mod_key in ti_cpc_pair_ideal_match and (mod_key[0] not in ban_set) and (mod_key[1] not in ban_set):
                merged_dict[query_key] = value        
        for key, value in rare_cpc_pair_dicts[j].items():
            if value <= 1:
                continue
            first, second = key.split()
            mod_key = (first[4:], second[4:])
            query_key = '(' + first + ' ' + second + ')'
            if mod_key in cpc_pair_ideal_match:
                merged_dict[query_key] = value
               
        for key, value in rare_ab_cpc_pair_dicts[j].items():
            if value <= 1:
                continue
            first, second = key.split()
            mod_key = (first[3:], second[4:])
            query_key = '(' + first + ' ' + second + ')'
            if mod_key in ab_cpc_pair_ideal_match  and (mod_key[0] not in ban_set):
                merged_dict[query_key] = value
        for key, value in rare_cpc_dicts[j].items():
            if value <= 1:
                continue
            mod_key = key[4:]
            if mod_key in cpc_ideal_match:
                merged_dict[key] = value
             
        for key, value in rare_ti_dicts[j].items():
            if value <= 1:
                continue
            if ' ' in key:
                mod_key = key.split('"')[1]
                first, second = mod_key.split()
                if not ((ti_freqs[first] not in ban_set) and (ti_freqs[second] not in ban_set)):
                    continue
            else:
                mod_key = key[3:]     
            if mod_key in ti_ideal_match and (mod_key not in ban_set):
                merged_dict[key] = value
                

        
        for key, value in rare_ab_dicts[j].items():
            if value <= 1:
                continue
            if ' ' in key:
                mod_key = key.split('"')[1]
                if mod_key.split()[0] in ban_set or mod_key.split()[1] in ban_set:
                    continue
            else:
                mod_key = key[3:]
                if mod_key in ban_set:
                    continue
                
            if mod_key in ab_ideal_match:
                merged_dict[key] = value
        
        for key, value in rare_clm_dicts[j].items():
            if value <= 2:
                continue
            if ' ' in key:
                mod_key = key.split('"')[1]
                if mod_key.split()[0] in ban_set or mod_key.split()[1] in ban_set:
                    continue
            else:
                mod_key = key[4:]
                if mod_key in ban_set:
                    continue
                
            if mod_key in clm_ideal_match:
                merged_dict[key] = value
        
        for key, value in rare_desc_dicts[j].items():
            if value <= 3:
                continue
            if ' ' in key:
                mod_key = key.split('"')[1]
                if mod_key.split()[0] in ban_set or mod_key.split()[1] in ban_set:
                    continue
            else:
                mod_key = key[5:]
                if mod_key in ban_set:
                    continue
                
            if mod_key in desc_ideal_match:
                merged_dict[key] = value
        if len(merged_dict) > max_len:
            max_len = len(merged_dict)
        merged_list.append(merged_dict)

    cleaned_merged_list = []
    for d in merged_list:
        to_append = list(dict(sorted(d.items(), key=lambda item: item[1], reverse=True)).keys())
        cleaned_merged_list.append(to_append)
    print(max_len)
    
    query = ''
    
    if max_len < 700:
        pairs_mscr_list = []
        for d in cleaned_merged_list:
            new_d = set()
            for key1 in d:
                for key2 in d:
                    if key1 < key2:
                        if '"' in key1 or '"'  in key2:
                            new_d.add((key1, key2))
                            continue
                        
                        if ' ' in key1 and ' ' in key2:
                            f1, f2 = key1.split()
                            s1, s2 = key2.split()
                            if f1 == s1:
                                new_d.add((key1, s2 + ')'))
                            elif f1[1:] == s2[:-1]:
                                new_d.add((key1, s1+ ')'))
                            elif f2[:-1] == s1[1:]:
                                new_d.add((key1, s2+ ')'))
                            elif f2 == s2:
                                new_d.add((key1, s1+ ')'))
                            else:
                                new_d.add((key1, key2))
                        
                        elif ' ' in key1 and ' ' not in key2:
                            
                            f1, f2 = key1.split()
                            if f1[1:] == key2 or f2[:-1] == key2:
                                #continue
                                new_d.add((key1, ''))
                            else:
                                new_d.add((key1, key2))
                        elif ' ' in key2 and ' ' not in key1:
                            f1, f2 = key2.split()
                            if f1[1:] == key1 or f2[:-1] == key1:
                                #continue
                                new_d.add((key2, ''))
                            else:
                                new_d.add((key1, key2))
                        else:
                            new_d.add((key1, key2))
                    elif key2 < key1:
                        if '"' in key1 or '"'  in key2:
                            new_d.add((key1, key2))
                            continue
                        
                        if ' ' in key1 and ' ' in key2:
                            f1, f2 = key1.split()
                            s1, s2 = key2.split()
                            if f1 == s1:
                                new_d.add((key1, s2 + ')'))
                            elif f1[1:] == s2[:-1]:
                                new_d.add((key1, s1+ ')'))
                            elif f2[:-1] == s1[1:]:
                                new_d.add((key1, s2+ ')'))
                            elif f2 == s2:
                                new_d.add((key1, s1+ ')'))
                            else:
                                new_d.add((key1, key2))
                        
                        elif ' ' in key1 and ' ' not in key2:
                            
                            f1, f2 = key1.split()
                            if f1[1:] == key2 or f2[:-1] == key2:
                                #continue
                                new_d.add((key1, ''))
                            else:
                                new_d.add((key1, key2))
                        elif ' ' in key2 and ' ' not in key1:
                            f1, f2 = key2.split()
                            if f1[1:] == key1 or f2[:-1] == key1:
                                #continue
                                new_d.add((key2, ''))
                            else:
                                new_d.add((key1, key2))
                        else:
                            new_d.add((key2, key1))
            pairs_mscr_list.append(new_d)

            
        found = 0
        min_set = mscr_mod(pairs_mscr_list,0)#mscr_new(cleaned_merged_list,0)
        #min_set = list(postprocess(min_set))
        min_set = list(min_set)
        for el in min_set:
            query += '(' + el[0] + ' ' + el[1] + ') OR '
            if len(query.split()) > 48:
                break
        
        if len(query.split()) < 48:
            if query != '':
                if query[-4:] == ' OR ':
                    query = query[:-4]
            query = '(' + query + ') OR '
            
        greedy_set = mscr_greedy(pairs_mscr_list, len(query.split()), min_set)
        
        for el in greedy_set:
            query += '(' + el[0] + ' ' + el[1] + ') OR '
            if len(query.split()) > 48:
                break
        
            
    else:
        min_set = mscr_new(cleaned_merged_list, 0)
        query += '(' + ' OR '.join(min_set) + ')'
        if len(query.split()) < 45:
            query = '(' + query + ') OR '
            greedy_set = mscr_greedy(cleaned_merged_list, len(query.split()), min_set)
            for s in greedy_set:
                query += s + ' OR '
                if len(query.split()) > 48:
                    break

    print(query)
    
    results.append(
        {"publication_number": test[i, "publication_number"], "query": query}
    )
    '''
    counter = 0

    score=0
    start = time.time()
    cand = execute_query(query, qp, searcher, target)
    print('candidates', len(cand))
    
    for t in range(50 - len(cand)):
        cand.append('null')
    print('tokens:', whoosh_utils.count_query_tokens(query))
    ap50_score = ap50(cand, target)
    inter = count_intersection(cand, target)
    print('inter', len(inter))
    #print(cand[0])
    if ap50_score > score:
        score = ap50_score
        print('score', score)
    scores.append(score)
    #print(time.time() - st)
    #break'''
    
#print("Average Score:", sum(scores) / len(scores))

  0%|          | 0/10 [00:00<?, ?it/s]

504


 10%|█         | 1/10 [00:31<04:41, 31.30s/it]

((ab:spectrometry ab:"mass spectrometry") OR (detd:importance detd:yield) OR ((ab:mass cpc:G01N33/6848) ) OR (detd:"data relating" cpc:G01N33/6842) OR (ab:sample detd:successful) OR (detd:"similar purpose" clm:minutes) OR (ti:chemical (ti:quantification ab:more))) OR (ab:"mass spectrometry" ab:spectrometry) OR ((ab:spectrometry cpc:G01N33/6848) (ab:mass)) OR ((ab:mass cpc:G01N33/6848) ab:spectrometry) OR ((ab:spectrometry cpc:G01N33/6848) ) OR (ab:spectrometry cpc:G01N33/6848) OR (ab:"mass spectrometry" (ab:spectrometry cpc:G01N33/6848)) OR 
447


 20%|██        | 2/10 [00:50<03:11, 23.93s/it]

...

(detd:rounded ti:paper) OR ((ti:holder ab:holder) ) OR (detd:tab detd:tabs) OR (ti:"loose leaf" ti:leaf) OR ((ti:clip cpc:Y10T24/206) ) OR (detd:formations ti:holder) OR (detd:"terminal end" detd:"being bent") OR (detd:"placed top" detd:"two members") OR (cpc:B42F13/006 detd:rounded) OR (detd:slipping detd:"planes substantially") OR (detd:"spaced parallel" detd:tab) OR (detd:"ends legs" ti:binder) OR (detd:fasteners (ti:fastener cpc:B42F13/08)) OR 
240


 90%|█████████ | 9/10 [01:46<00:08,  8.82s/it]

(((ti:reader ab:reader) ) OR ((ti:optical ab:optical) ) OR (detd:binary detd:rows) OR (ab:read ab:reading) OR (ab:scanning ab:scans) OR ((ab:each cpc:G06K7/10831) (ab:than)) OR (detd:converter detd:sensed) OR ((ti:image ab:signal) ti:terminal))) OR (detd:"overcome problems" detd:"problems associated") OR ((ab:device cpc:G07D7/121) ab:bank) OR (ab:scanning clm:capturing) OR (detd:encodes ab:"leading edge")) OR ((ti:optical ab:light) ) OR 


100%|██████████| 10/10 [01:50<00:00, 11.09s/it]

42
((ti:acetylene ti:"acetylene gas") OR (detd:snugly detd:"limit its") OR (detd:"engage said" detd:"directly below") OR (detd:"hereby declare" detd:"others skilled") OR ((cpc:G06F11/0715 cpc:C10H5/00) detd:tn) OR (detd:"inwardly projecting" detd:"device carried") OR (detd:"end secured" detd:"downwardly against") OR (detd:"vertical cylinder" detd:"others skilled")) OR ((ti:acetylene ti:gas) ) OR (ti:"acetylene gas" ti:acetylene) OR ((ti:acetylene ti:gas) ti:"acetylene gas") OR 


In [34]:
# Remove unwanted files and directories that may cause submission errors
!rm -rf /kaggle/working/*

In [35]:
submission = pl.DataFrame(results)
submission.write_csv("submission.csv")

submission

publication_number,query
str,str
"""US-2017082634-A1""","""((ab:spectrometry ab:""mass spe…"
"""US-2017180470-A1""","""((detd:""computing device"" detd…"
"""US-2018029544-A1""","""((cpc:Y02E10/50 OR ti:battery …"
"""US-2022408153-A1""","""((detd:family OR detd:""informa…"
"""US-2268569-A""","""(cpc:E02F3/3486 cpc:E02F9/022)…"
"""US-3371854-A""","""(ti:vacuum ti:""vacuum pump"") O…"
"""US-3589189-A""","""((ti:meter cpc:G01F3/18) ) OR …"
"""US-3881203-A""","""(detd:rounded ti:paper) OR ((t…"
"""US-4845770-A""","""(((ti:reader ab:reader) ) OR (…"
